In [120]:
import pandas as pd 

lab_a = pd.read_csv('Invitro.csv')
lab_b = pd.read_csv('OpenHealth.csv')
lab_c = pd.read_csv('Invivo.csv')

lab_a['Лаборатория'] = 'Invitro'
lab_b['Лаборатория'] = 'OpenHealth'
lab_c['Лаборатория'] = 'Invivo'

added the column with name of laboratory to dataframes

In [121]:
all_labs = pd.concat([lab_a, lab_b, lab_c], ignore_index=True)

united all labs into one dataframe

In [122]:
all_labs.columns

Index(['Код', 'Наименование', 'Биоматериал', 'Тип результата',
       'Лаборатория исполнения', 'Алматы, СРОК', 'Партнерская цена, в тенге.',
       'Цена для клиентов на сайте checkdoc', 'Лаборатория',
       'Перечень исследований', 'Розница', 'Партнерская', 'Тип контейнера',
       'Срок готовности', 'Цена', 'Цена со скидкой', 'Сроки выполнения'],
      dtype='object')

In [123]:
all_labs['Партнерская цена'] = all_labs['Партнерская'].combine_first(all_labs['Партнерская цена, в тенге.'])
all_labs['Наименование анализа'] = all_labs['Наименование'].combine_first(all_labs['Перечень исследований'])

all_labs.drop(columns='Партнерская', inplace=True)
all_labs.drop(columns='Партнерская цена, в тенге.', inplace=True)
all_labs.drop(columns='Наименование', inplace=True)
all_labs.drop(columns='Перечень исследований', inplace=True)


так как есть две разных колонки "партнерская" и "партнерская цена в тенге", которые хранят значения для разных лабораторий, объединила их в одну колонку - "партнерская цена". Она работает по принципу если есть значение из первой колонки, то оставляю его, если нет, то ставлю то, что во второй. Тоже самое с наименованием, а ненужные столбцы удаляем

In [124]:
all_labs["Наименование анализа"].value_counts()

Наименование анализа
Мочевина                                                                                                                                                                                                                                                              3
Витамин С (аскорбиновая кислота)                                                                                                                                                                                                                                      3
Тромбиновое время                                                                                                                                                                                                                                                     3
АПТВ (активированное парциальное тромбопластинновое время)                                                                                                                                 

In [125]:
def is_section_header(row):
    name = str(row['Наименование анализа'])
    return (
        name.isupper() and
        pd.isna(row['Код']) and
        pd.isna(row.get('Партнерская цена', None)) and
        pd.isna(row.get('Цена', None)) and
        pd.isna(row.get('Цена для клиентов на сайте checkdoc', None))
    )

all_labs['is_header'] = all_labs.apply(is_section_header, axis=1)


так как в изначальном документе в таблицах OpenHealth и Invivo классификация по категориям проводилась буквально перед всеми анализами (капс в наименовании), относящимся к этой категории, сейчас компьютер думает, что название категории это название еще одного анализа. То есть мы написали функцию, которая выбирает столбец Наименование, ищет название написаное капсом и при этом чтобы остальные строки были NaN. Так как мы указали axis=1, то он будет применять эту функцию к каждой строке и искать название секции. Apply() возвращает True, если всё что написано в теле функции окажется правдой 

In [126]:
category_col = []
current_category = None

for is_header, name in zip(all_labs['is_header'], all_labs['Наименование анализа']):
    if is_header:
        current_category = name.title() 
        category_col.append(None)  
    else:
        category_col.append(current_category)

all_labs['Категория'] = category_col

мы создаем пустой лист, куда будем складывать значения категорий для каждого из анализов (то есть построчно). Так же ставим значение текущей категории дефолтное - NaN. В цикле берем булевые значения (категория/не категория) и само наименование анализов. Если категория, то меняем значение текущей категории на это название (но уже не капсом), при этом в лист категорий добавляем NaN, потому что потом мы просто уберем эту строку (так как там лежит не наименование анализа и она нам не нужна). Но если это не категория, то тогда мы добавляем в список категорий название текущей категории, чтобы потом у этого анализа была именна эта категория. В конце уже создаем новую колонку категория, которой присваеваем значения списка

In [127]:
all_labs = all_labs[~all_labs['is_header']].copy()
all_labs.drop(columns='is_header', inplace=True)

`~all_labs['is_header']` тоже самое что НЕ КАТЕГОРИИ, то есть в датафрейме all_labs мы оставляем копию дата фрейма all_labs где в колонке "is_header" лежат не категории (то есть чистим от просто названий категорий)

а дальше уже просто удаляем и саму колонку "is_header", потому что она нам не нужна больше

In [128]:
all_labs.sample(15)

,Код,Биоматериал,Тип результата,Лаборатория исполнения,"Алматы, СРОК",Цена для клиентов на сайте checkdoc,Лаборатория,Розница,Тип контейнера,Срок готовности,Цена,Цена со скидкой,Сроки выполнения,Партнерская цена,Наименование анализа,Категория
1354,7725C1NHI,Цельная кровь,Описание результата врачом - генетиком,МСК,до 16,354 984,Invitro,NaN,NaN,NaN,NaN,NaN,NaN,253 560,Наследственный ангионевротический отек. Поиск ...,None
650,80,Сыворотка крови,Кол,Алматы,до 2,3 450,Invitro,NaN,NaN,NaN,NaN,NaN,NaN,2 464,Антитела класса IgG к Тoxoplasma gondii (Аnti-...,None
1753,6189АБ,Сыворотка крови,0,МСК,до 4,3 461,Invitro,NaN,NaN,NaN,NaN,NaN,NaN,2 472,"Одуванчик (Taraxacum vulgare) (w8), IgE",None
2413,96-011,NaN,NaN,NaN,NaN,49700.0,OpenHealth,40000.0,Nasopharyngeal swab,1-2,NaN,NaN,NaN,35500.0,Панель Респираторного Вируса/Бактерия (26 Тип...,Гематологические Исследования
1967,NaN,NaN,NaN,NaN,NaN,0.0,OpenHealth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cпецифические белки (Specific proteins),Гематологические Исследования
81,295,Сыворотка крови,Кол,МСК,до 8,6 406,Invitro,NaN,NaN,NaN,NaN,NaN,NaN,4 576,Пепсиноген II (Pepsinogen II),None
2113,99-124,NaN,NaN,NaN,NaN,13720.0,OpenHealth,14000.0,Serum,10-12,NaN,NaN,NaN,9800.0,АТ к микросомальной фракции печени и почек (LKM),Гематологические Исследования
146,1198,Сыворотка крови,Кол,МСК,до 2,13 496,Invitro,NaN,NaN,NaN,NaN,NaN,NaN,9 640,Белок S100 (S100 Рrotein),None
830,813,Сыворотка крови,Полукол,МСК,до 13,11 144,Invitro,NaN,NaN,NaN,NaN,NaN,NaN,7 960,Антитела класса IgG к десмосомам эпидермиса (D...,None
3802,12-134,NaN,NaN,NaN,NaN,NaN,Invivo,NaN,NaN,NaN,"4,9",3430.0,4-6,NaN,Бак. посев вагинального содержимого на золотис...,Микробиологические Исследования


In [ ]:
all_labs['Партнерская цена'] = all_labs['Партнерская цена'].combine_first(all_labs['Цена со скидкой'])
all_labs.drop(columns="Цена со скидкой", inplace=True)